In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("retail_store_sales.csv")

In [5]:
!pip install ydata-profiling

In [6]:
!pip install --upgrade ipywidgets
!pip install --upgrade ydata-profiling

In [7]:
from ydata_profiling import ProfileReport

profile = ProfileReport(
    df,
    explorative=True,
    minimal=True
)

profile.to_file("FULL_DATAFRAME_INDIVAL_REPORTS.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 59.74it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
import os
os.path.exists("FULL_DATAFRAME_INDIVAL_REPORTS.html")


True

In [9]:
import webbrowser
webbrowser.open("FULL_DATAFRAME_INDIVAL_REPORTS.html")

True

In [10]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12575 entries, 0 to 12574
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    12575 non-null  object 
 1   Customer ID       12575 non-null  object 
 2   Category          12575 non-null  object 
 3   Item              11362 non-null  object 
 4   Price Per Unit    11966 non-null  float64
 5   Quantity          11971 non-null  float64
 6   Total Spent       11971 non-null  float64
 7   Payment Method    12575 non-null  object 
 8   Location          12575 non-null  object 
 9   Transaction Date  12575 non-null  object 
 10  Discount Applied  8376 non-null   object 
dtypes: float64(3), object(8)
memory usage: 1.1+ MB
None


In [11]:
# This line checks how many missing (empty) values are in each column of your DataFrame.
df.isnull().sum()

Transaction ID         0
Customer ID            0
Category               0
Item                1213
Price Per Unit       609
Quantity             604
Total Spent          604
Payment Method         0
Location               0
Transaction Date       0
Discount Applied    4199
dtype: int64

In [12]:
df.isnull().sum().sum()

7229

In [13]:
# STEP 1 — Fix numeric columns first (NON-NEGOTIABLE)
#These 3 columns are mathematically linked:
# Total Spent = Price Per Unit × Quantity

In [14]:
# 1A. Enforce numeric type

num_cols = ['Price Per Unit', 'Quantity', 'Total Spent']

for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

    

#This line does the real work:
#pd.to_numeric(...) tries to convert the column values into numbers
#errors='coerce' means:
#If a value can’t be converted (like "abc", "N/A", or "--"),
#Pandas will replace it with NaN instead of throwing an error

In [15]:
# 1B. Recover missing Total Spent

#df['Total Spent'].isna()
#→ Total Spent is missing (NaN)

#df['Price Per Unit'].notna()
#→ Price Per Unit is available

#df['Quantity'].notna()
#→ Quantity is available

# “Select rows where Total Spent is empty, but Price Per Unit and Quantity are present.”

mask = df['Total Spent'].isna() & df['Price Per Unit'].notna() & df['Quantity'].notna()
df.loc[mask, 'Total Spent'] = df.loc[mask, 'Price Per Unit'] * df.loc[mask, 'Quantity']



# This line does the calculation only for the rows selected by the mask.
# df.loc[mask, 'Price Per Unit'] → gets the price
# df.loc[mask, 'Quantity'] → gets the quantity
# Multiply them
# Store the result back into Total Spent




# Very important
# ✔ Company-grade logic

In [29]:
# 1C. Recover missing Price Per Unit
mask = (
    df['Price Per Unit'].isna()
    & df['Total Spent'].notna()
    & df['Quantity'].notna()
)

df.loc[mask, 'Price Per Unit'] = (
    df.loc[mask, 'Total Spent'] / df.loc[mask, 'Quantity']
)


In [33]:
# 1D. Recover missing Quantity
mask = df['Quantity'].isna() & df['Total Spent'].notna() & df['Price Per Unit'].notna()
df.loc[mask, 'Quantity'] = df.loc[mask, 'Total Spent'] / df.loc[mask, 'Price Per Unit']

# 🧠 This alone will remove most of those 600 missing values.

In [35]:
# STEP 2 — Clean & Recover Item (24% missing)

In [37]:
# 2A. Standardize garbage                                    

df['Item'] = df['Item'].replace(                           
    ['ERROR', 'UNKNOWN', 'Unknown', 'error', '', ' '],
    pd.NA
)
# pd.NA is Pandas’ official way of saying:
# “This value is missing or unknown.”
#  convert into NAN in item colum

In [39]:
# 2B. Recover Item using Category

item_mode_by_category = df.groupby('Category')['Item'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else pd.NA)
df['Item'] = df['Item'].fillna(df['Category'].map(item_mode_by_category))

# df.groupby('Category')
#Groups the DataFrame by the Category column.

#['Item']
#Focuses only on the Item column within each category.

#.agg(lambda x: ...)
#Applies a custom function to each group of Item values.

#x.mode()
#Finds the most frequent value(s) in the group.

#x.mode().iloc[0]
#If there are multiple modes, it picks the first one.

#if not x.mode().empty else pd.NA
#If a category has no mode (e.g., all values are missing), return pd.NA.

#✅ Result:
#item_mode_by_category is a Series where:

#index = Category

#value = most common Item for that category


# What it does:

#df['Category'].map(item_mode_by_category)
#Replaces each category with its most common Item (from step 1).

#fillna(...)
#Fills missing values in Item using that mapped value.

#So:
#If Item is missing (NaN)
#It gets replaced by the most frequent Item in the same Category

#✔ Real retail logic
#✔ No guessing
#✔ Interview-approved

In [41]:
# 2C. Label unresolved items
df['Item'] = df['Item'].fillna('UNRESOLVED_ITEM')


In [43]:
#STEP 3 — Fix Discount Applied (84% missing)
#This column is binary, not analytical numeric data.


In [45]:
#🔹 Company decision:
#If discount info is missing → assume NO discount

df['Discount Applied'] = df['Discount Applied'].fillna(False).astype('boolean')

#✔ Clean
#✔ Business-safe
#✔ Dashboard-friendly

In [47]:
# STEP 4 — Final validation (MANDATORY)

In [49]:
df.isna().sum()



Transaction ID        0
Customer ID           0
Category              0
Item                  0
Price Per Unit        0
Quantity            604
Total Spent         604
Payment Method        0
Location              0
Transaction Date      0
Discount Applied      0
dtype: int64

In [51]:
df['Item'].isna().sum()

0

In [53]:
# 1️⃣ Recover Price Per Unit
mask = df['Price Per Unit'].isna() & df['Total Spent'].notna() & df['Quantity'].notna()
df.loc[mask, 'Price Per Unit'] = df.loc[mask, 'Total Spent'] / df.loc[mask, 'Quantity']


In [55]:
# 2️⃣ Recover Total Spent
mask = df['Total Spent'].isna() & df['Price Per Unit'].notna() & df['Quantity'].notna()
df.loc[mask, 'Total Spent'] = df.loc[mask, 'Price Per Unit'] * df.loc[mask, 'Quantity']


In [57]:
# 3️⃣ Recover Quantity (rounding is important)
mask = df['Quantity'].isna() & df['Total Spent'].notna() & df['Price Per Unit'].notna()
df.loc[mask, 'Quantity'] = (df.loc[mask, 'Total Spent'] / df.loc[mask, 'Price Per Unit']).round()


In [59]:
df.isna().sum()


Transaction ID        0
Customer ID           0
Category              0
Item                  0
Price Per Unit        0
Quantity            604
Total Spent         604
Payment Method        0
Location              0
Transaction Date      0
Discount Applied      0
dtype: int64

In [65]:
# Alternative (bulk, very clean)
df = df.fillna({
    'Quantity': df['Quantity'].median(),
    'Total Spent': df['Total Spent'].median()
})
# Used in pipelines & ETL jobs.import os
os.getcwd()


In [67]:
df.isna().sum()

Transaction ID      0
Customer ID         0
Category            0
Item                0
Price Per Unit      0
Quantity            0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
Discount Applied    0
dtype: int64

In [73]:
df.to_csv("cleaned_data.csv", index=False)


In [75]:
import os
os.getcwd()


'C:\\Users\\Sorav\\OneDrive\\Desktop\\Messy_Sales_Data'

In [77]:
df.to_csv(r"C:\PowerBI_Data\cleaned_sales_data.csv", index=False)


OSError: Cannot save file into a non-existent directory: 'C:\PowerBI_Data'